# **Instalações e Bibliotecas**

In [1]:
%%capture

!pip install shap
!pip install optuna

In [2]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

# **Funções**

O arquivo 'functions_pred_cruzada' contém as funções que serão utilizadas para selecionar, pré-processar e treinar e validar os modelos de machine learning

In [3]:
!gdown 1uVoArwRTJmvbyFLyFniwFvWGHMfiNIWO --quiet

from functions_pred_cruzada import *

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# **Dados**

In [5]:
# Dados brutos com todos os tipos de câncer
!gdown 1P19f3kA5_s4sv5Pg6D6UKwwQvZvaAnQT --quiet

df_geral = pd.read_csv('pacigeral.csv')
print(df_geral.shape)
df_geral.head(3)

Columns (35,36,73,74,75,78,89,90) have mixed types. Specify dtype option on import or set low_memory=False.


(1134043, 100)


,ESCOLARI,IDADE,SEXO,UFNASC,UFRESID,IBGE,CIDADE,CATEATEND,DTCONSULT,CLINICA,...,REC03,REC04,IBGEATEN,CIDO,DSCCIDO,HABILIT,HABIT11,HABILIT1,HABILIT2,CIDADEH
0,2,100,2,SP,SP,3550308,SAO PAULO,9,2000-06-09,3,...,NaN,NaN,3550308,80703,CARCINOMA ESCAMOCELULAR SOE,7,CACON com Servio de Oncologia Pedi trica,3,2,SÆo Paulo
1,1,102,1,MG,MG,3133402,ITAPAGIPE,9,2002-04-29,24,...,NaN,NaN,3505500,80903,CARCINOMA BASOCELULAR SOE,7,CACON com Servio de Oncologia Pedi trica,3,2,Barretos
2,2,101,2,MG,SP,3543402,RIBEIRAO PRETO,9,2000-12-03,10,...,NaN,NaN,3543402,80703,CARCINOMA ESCAMOCELULAR SOE,7,CACON com Servio de Oncologia Pedi trica,3,2,RibeirÆo Preto


**Tipos com maior incidência**

In [ ]:
df_geral.TOPOGRUP.value_counts(normalize=True).head(15)

C44    0.229273
C50    0.136342
C61    0.103496
C34    0.047272
C53    0.046387
C18    0.037844
C42    0.037599
C16    0.036821
C20    0.028988
C73    0.023747
C77    0.018756
C15    0.018305
C67    0.017882
C32    0.015979
C64    0.015855
Name: TOPOGRUP, dtype: float64

*   C44 - Pele 22,93%
*   C50 - Mama 13,63%
*   C61 - Próstata 10,35%
*   C34 - Pulmão 4,73%
*   C53 - Colo de Útero 4,64%
*   C18 - Cólon 3,78%
*   C42 - ?? 3,76%
*   C16 - Estômago 3,68%
*   C20 - Reto 2,90%
*   C73 - Tiróide 2,37%
*   C77 - ?? 1,88%
*   C15 - Esôfago 1,83%
*   C67 - Bexiga 1,79%
*   C32 - Laringe 1,60%
*   C64 - Rim 1,59%



**Tipos com maior incidência por sexo**

In [ ]:
df_masc = df_geral[df_geral.SEXO == 1]
df_fem = df_geral[df_geral.SEXO == 2]

print(df_masc.shape)
print(df_fem.shape)

(564295, 100)
(569748, 100)


In [ ]:
df_masc.TOPOGRUP.value_counts(normalize=True).head(10)

C44    0.242235
C61    0.207992
C34    0.057615
C16    0.047716
C42    0.040686
C18    0.037027
C20    0.032261
C15    0.030612
C32    0.028001
C67    0.026417
Name: TOPOGRUP, dtype: float64

*   C44 - Pele 24,22%
*   C61 - Próstata 20,80%
*   C34 - Pulmão 5,76%
*   C16 - Estômago 4,77%
*   C42 - ?? 4,07%
*   C18 - Cólon 3,70%
*   C20 - Reto 3,22%
*   C15 - Esôfago 3,06%
*   C32 - Laringe 2,80%
*   C67 - Bexiga 2,64%

In [ ]:
df_fem.TOPOGRUP.value_counts(normalize=True).head(10)

C50    0.269551
C44    0.216434
C53    0.092330
C18    0.038654
C73    0.037580
C34    0.037027
C42    0.034542
C54    0.029424
C16    0.026031
C20    0.025746
Name: TOPOGRUP, dtype: float64

*   C50 - Mama 26,95%
*   C44 - Pele 21,64%
*   C53 - Colo de Útero 9,23%
*   C18 - Cólon 3,86%
*   C73 - Tiróide 3,75%
*   C34 - Pulmão 3,70%
*   C42 - ?? 3,45%
*   C54 - Corpo do Útero 2,94%
*   C16 - Estômago 2,60%
*   C20 - Reto 2,57%

## **Colorretal**

Seleção dos dados de câncer colorretal

In [ ]:
# Colorretal - C18, C19, C20
colorretal = ['C18', 'C19', 'C20']

df_colo = data_prep(df_geral, colorretal)
print(df_colo.shape)
df_colo.head(3)

(60887, 30)


,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,TOPOGRUP,EC,TRATHOSP,NENHUM,...,DIAGTRAT,ANODIAG,DRS,RRAS,IBGEATEN,HABILIT2,ULTIDIAG,ESCOLARI_preench,IBGE_idem_IBGEATEN,presenca_rec
30913,28,2,3548708,9,1,C180,C18,III,E,0,...,0,2003,01,1,3550308,1,415,3.0,0,0
30914,26,2,3550308,9,1,C180,C18,III,A,0,...,0,2001,01,6,3550308,2,2478,2.0,1,0
30916,22,2,3503208,9,2,C180,C18,IV,C,0,...,0,2001,03,13,3503208,1,255,4.0,1,0


In [ ]:
df_colo.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'ULTINFO', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT',
       'ANODIAG', 'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ULTIDIAG',
       'ESCOLARI_preench', 'IBGE_idem_IBGEATEN', 'presenca_rec'],
      dtype='object')

Criação da coluna de saída

In [ ]:
df_colo = get_labels(df_colo)
df_colo.shape

(60887, 32)

In [ ]:
df_colo.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG',
       'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ESCOLARI_preench',
       'IBGE_idem_IBGEATEN', 'presenca_rec', 'obito_geral', 'sobrevida_ano1',
       'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

Salvando os dados no Drive

In [ ]:
df_colo.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Predição cruzada/Datasets/colorretal.csv',
               encoding='utf-8', index=False)

## **Pulmão**

Seleção dos dados de câncer de pulmão

In [ ]:
# Pulmão - C34
pulmao = ['C34']

df_pulmao = data_prep(df_geral, pulmao)
print(df_pulmao.shape)
df_pulmao.head(3)

(38646, 30)


,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,TOPOGRUP,EC,TRATHOSP,NENHUM,...,DIAGTRAT,ANODIAG,DRS,RRAS,IBGEATEN,HABILIT2,ULTIDIAG,ESCOLARI_preench,IBGE_idem_IBGEATEN,presenca_rec
32294,28,2,3520103,9,1,C340,C34,IIIB,J,1,...,99,2001,08,13,3505500,2,64,2.0,0,0
32295,28,2,3554508,9,1,C340,C34,IIIB,C,0,...,0,2010,16,8,3525300,2,1006,3.0,0,1
32298,27,2,3507001,2,2,C340,C34,IV,F,0,...,0,2013,16,8,3525300,2,59,4.0,0,0


In [ ]:
df_pulmao.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'ULTINFO', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT',
       'ANODIAG', 'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ULTIDIAG',
       'ESCOLARI_preench', 'IBGE_idem_IBGEATEN', 'presenca_rec'],
      dtype='object')

Criação da coluna de saída

In [ ]:
df_pulmao = get_labels(df_pulmao)
df_pulmao.shape

(38646, 32)

In [ ]:
df_pulmao.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG',
       'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ESCOLARI_preench',
       'IBGE_idem_IBGEATEN', 'presenca_rec', 'obito_geral', 'sobrevida_ano1',
       'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

Salvando os dados no Drive

In [ ]:
df_pulmao.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Predição cruzada/Datasets/pulmao.csv',
                 encoding='utf-8', index=False)

## **Mama**

Seleção dos dados de câncer de mama

In [ ]:
# Mama - C50
mama = ['C50']

df_mama = data_prep(df_geral, mama)
print(df_mama.shape)
df_mama.head(3)

(117338, 30)


,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,TOPOGRUP,EC,TRATHOSP,NENHUM,...,DIAGTRAT,ANODIAG,DRS,RRAS,IBGEATEN,HABILIT2,ULTIDIAG,ESCOLARI_preench,IBGE_idem_IBGEATEN,presenca_rec
7,102,2,3550308,9,1,C509,C50,IIIB,D,0,...,2,2001,01,6,3550308,2,436,3.0,1,1
39363,28,2,3547809,9,2,C500,C50,I,G,0,...,2,2002,01,1,3547809,1,7107,4.0,1,0
39364,27,2,3512001,2,2,C500,C50,I,H,0,...,0,2003,05,13,3505500,2,6587,4.0,0,1


In [ ]:
df_mama.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'ULTINFO', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT',
       'ANODIAG', 'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ULTIDIAG',
       'ESCOLARI_preench', 'IBGE_idem_IBGEATEN', 'presenca_rec'],
      dtype='object')

Criação da coluna de saída

In [ ]:
df_mama = get_labels(df_mama)
df_mama.shape

(117338, 32)

In [ ]:
df_mama.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG',
       'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ESCOLARI_preench',
       'IBGE_idem_IBGEATEN', 'presenca_rec', 'obito_geral', 'sobrevida_ano1',
       'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

Salvando os dados no Drive

In [ ]:
df_mama.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Predição cruzada/Datasets/mama.csv',
               encoding='utf-8', index=False)

## **Colo de Útero**

Seleção dos dados de câncer de colo de útero



In [ ]:
# Colo de Útero - C53
colo_utero = ['C53']

df_colo_utero = data_prep(df_geral, colo_utero)
print(df_colo_utero.shape)
df_colo_utero.head(3)

(23306, 30)


,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,TOPOGRUP,EC,TRATHOSP,NENHUM,...,DIAGTRAT,ANODIAG,DRS,RRAS,IBGEATEN,HABILIT2,ULTIDIAG,ESCOLARI_preench,IBGE_idem_IBGEATEN,presenca_rec
41507,27,2,3548906,9,2,C530,C53,IIIB,B,0,...,1,2002,03,13,3503208,1,679,4.0,0,0
41510,29,2,3518701,9,1,C530,C53,IB2,E,0,...,0,2008,04,7,3518701,1,671,3.0,1,0
41514,29,2,3522109,2,2,C530,C53,IB1,A,0,...,1,2011,04,7,3550308,2,4120,3.0,0,1


In [ ]:
df_colo_utero.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'ULTINFO', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT',
       'ANODIAG', 'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ULTIDIAG',
       'ESCOLARI_preench', 'IBGE_idem_IBGEATEN', 'presenca_rec'],
      dtype='object')

Criação da coluna de saída

In [ ]:
df_colo_utero = get_labels(df_colo_utero)
df_colo_utero.shape

(23306, 32)

In [ ]:
df_colo_utero.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG',
       'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ESCOLARI_preench',
       'IBGE_idem_IBGEATEN', 'presenca_rec', 'obito_geral', 'sobrevida_ano1',
       'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

Salvando os dados no Drive

In [ ]:
df_colo_utero.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Predição cruzada/Datasets/colo_utero.csv',
                     encoding='utf-8', index=False)

## **Próstata**

Seleção dos dados de câncer de próstata

In [ ]:
# Próstata - C61
prostata = ['C61']

df_prostata = data_prep(df_geral, prostata)
print(df_prostata.shape)
df_prostata.head(3)

(94106, 30)


,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,TOPOGRUP,EC,TRATHOSP,NENHUM,...,DIAGTRAT,ANODIAG,DRS,RRAS,IBGEATEN,HABILIT2,ULTIDIAG,ESCOLARI_preench,IBGE_idem_IBGEATEN,presenca_rec
49597,24,1,3524402,2,2,C619,C61,I,C,0,...,2,2011,17,17,3524402,1,4093,2.0,1,1
49600,22,1,3519204,2,2,C619,C61,I,J,1,...,99,2015,09,10,3555000,1,989,2.0,0,0
49601,20,1,3502804,2,1,C619,C61,II,B,0,...,0,2013,02,12,3502804,1,3135,2.0,1,1


In [ ]:
df_prostata.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'ULTINFO', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT',
       'ANODIAG', 'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ULTIDIAG',
       'ESCOLARI_preench', 'IBGE_idem_IBGEATEN', 'presenca_rec'],
      dtype='object')

Criação da coluna de saída

In [ ]:
df_prostata = get_labels(df_prostata)
df_prostata.shape

(94106, 32)

In [ ]:
df_prostata.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG',
       'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ESCOLARI_preench',
       'IBGE_idem_IBGEATEN', 'presenca_rec', 'obito_geral', 'sobrevida_ano1',
       'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

Salvando os dados no Drive

In [ ]:
df_prostata.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Predição cruzada/Datasets/prostata.csv',
                   encoding='utf-8', index=False)

## **Pele**

Seleção dos dados de câncer de pele

In [ ]:
teste = df_geral[df_geral.TOPOGRUP == 'C44']
teste.shape

(260005, 100)

In [ ]:
# Pele - C44 (Other malignant neoplasms of skin)
pele = ['C44']

df_pele = data_prep(df_geral, pele)
print(df_pele.shape)
df_pele.head(3)

(183164, 30)


,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,TOPOGRUP,EC,TRATHOSP,NENHUM,...,DIAGTRAT,ANODIAG,DRS,RRAS,IBGEATEN,HABILIT2,ULTIDIAG,ESCOLARI_preench,IBGE_idem_IBGEATEN,presenca_rec
2,101,2,3543402,9,1,C443,C44,I,I,0,...,0,2000,13,13,3543402,2,719,2.0,1,0
4,100,1,3522802,2,1,C443,C44,II,A,0,...,0,2000,06,9,3525300,2,428,1.0,0,0
6,105,2,3546603,9,1,C447,C44,I,A,0,...,0,2005,15,12,3505500,2,9,1.0,0,0


In [ ]:
df_pele.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'ULTINFO', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT',
       'ANODIAG', 'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ULTIDIAG',
       'ESCOLARI_preench', 'IBGE_idem_IBGEATEN', 'presenca_rec'],
      dtype='object')

Criação da coluna de saída

In [ ]:
df_pele = get_labels(df_pele)
df_pele.shape

(183164, 32)

In [ ]:
df_pele.columns

Index(['IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV', 'TOPO', 'TOPOGRUP',
       'EC', 'TRATHOSP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO', 'HORMONIO',
       'TMO', 'IMUNO', 'OUTROS', 'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG',
       'DRS', 'RRAS', 'IBGEATEN', 'HABILIT2', 'ESCOLARI_preench',
       'IBGE_idem_IBGEATEN', 'presenca_rec', 'obito_geral', 'sobrevida_ano1',
       'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

Salvando os dados no Drive

In [ ]:
df_pele.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Predição cruzada/Datasets/pele.csv',
               encoding='utf-8', index=False)

## **Informações**

In [4]:
def data_prep_v2(df, topogrup):
    df_aux = df.copy()

    # TOPOGRUP
    df_aux = df_aux[df_aux.TOPOGRUP.isin(topogrup)]

    # IDADE > 19
    df_aux = df_aux[df_aux.IDADE > 19]

    # Somente residentes do estado de SP
    df_aux = df_aux[df_aux.UFRESID == 'SP']

    # ECGRUP sem 0, X e Y
    df_aux = df_aux[~df_aux.ECGRUP.isin(['0','X','Y'])]

    # ANODIAG até 2019 e em categorias de 5 em 5 anos
    df_aux = df_aux[df_aux.ANODIAG < 2020]

    # Somente com confirmação microscópica
    df_aux = df_aux[df_aux.BASEDIAG == 3]

    # Tratamento das colunas com datas
    list_datas = ['DTCONSULT', 'DTDIAG', 'DTTRAT', 'DTULTINFO']

    for col_data in list_datas:
        df_aux[col_data] = pd.to_datetime(df_aux[col_data])

    df_aux['CONSDIAG'] = (df_aux.DTDIAG - df_aux.DTCONSULT).dt.days
    df_aux['DIAGTRAT'] = (df_aux.DTTRAT - df_aux.DTDIAG).dt.days
    df_aux['TRATCONS'] = (df_aux.DTTRAT - df_aux.DTCONSULT).dt.days
    df_aux['ULTIDIAG'] = (df_aux.DTULTINFO - df_aux.DTDIAG).dt.days
    df_aux[['DIAGTRAT', 'TRATCONS']] = df_aux[['DIAGTRAT', 'TRATCONS']].fillna(-1)

    df_aux.CONSDIAG = [0 if consdiag <= 30 else 1 if consdiag <= 60 else 2 for consdiag in df_aux.CONSDIAG]
    df_aux.TRATCONS = [99 if tratcons < 0 else 0 if tratcons <= 60 else 1 if tratcons <= 90 else 2 for tratcons in df_aux.TRATCONS]
    df_aux.DIAGTRAT = [99 if diagtrat < 0 else 0 if diagtrat <= 60 else 1 if diagtrat <= 90 else 2 for diagtrat in df_aux.DIAGTRAT]

    # Variável para pessoas que fazem tratamento na mesma cidade (0 = Não, 1 = Sim)
    df_aux['IBGE_idem_IBGEATEN'] = 0
    df_aux.loc[df_aux.IBGE == df_aux.IBGEATEN, 'IBGE_idem_IBGEATEN'] = 1

    # Variável para presença de recidiva (0 = Não, 1 = Sim)
    df_aux['presenca_rec'] = [0 if rec == 1 else 1 for rec in df_aux.RECNENHUM]
    df_aux.loc[(df_aux.presenca_rec == 0) & (df_aux.DTRECIDIVA.notnull()), 'presenca_rec'] = 1

    # Somente o número da DRS
    DRS_expand = df_aux.DRS.str.split(' ', expand=True)
    df_aux['DRS'] = DRS_expand[1]

    # Retirada das colunas que não serão usadas
    cols = df_aux.columns
    drop_cols = ['ESCOLARI', 'UFNASC', 'UFRESID', 'CIDADE', 'DTCONSULT', 'CLINICA',
                 'DTDIAG', 'BASEDIAG', 'DESCTOPO', 'MORFO', 'DESCMORFO', #'ECGRUP',
                 'T', 'N', 'M', 'PT', 'PN', 'PM', 'S', 'G', 'LOCALTNM', 'IDMITOTIC',
                 'PSA', 'GLEASON', 'OUTRACLA', 'META01', 'META02', 'META03',
                 'META04', 'DTTRAT', 'NAOTRAT', 'TRATAMENTO', 'TRATFANTES',
                 'TRATFAPOS', 'NENHUMANT', 'CIRURANT', 'RADIOANT', 'QUIMIOANT',
                 'HORMOANT', 'TMOANT', 'IMUNOANT', 'OUTROANT', 'NENHUMAPOS',
                 'CIRURAPOS', 'RADIOAPOS', 'QUIMIOAPOS', 'HORMOAPOS', 'TMOAPOS',
                 'IMUNOAPOS', 'OUTROAPOS', 'DTULTINFO', 'CICI', 'CICIGRUP',
                 'CICISUBGRU', 'FAIXAETAR', 'LATERALI', 'INSTORIG', 'PERDASEG',
                 'ERRO', 'DTRECIDIVA', 'RECNENHUM', 'RECLOCAL', 'RECREGIO',
                 'RECDIST', 'REC01', 'REC02', 'REC03', 'REC04', 'CIDO', 'DSCCIDO',
                 'HABILIT', 'HABIT11', 'HABILIT1', 'CIDADEH']

    cols = cols.drop(drop_cols)

    return df_aux[cols]

In [6]:
# Colorretal - C18, C19, C20
colorretal = ['C18', 'C19', 'C20']

df_colo = data_prep_v2(df_geral, colorretal)
print(df_colo.shape)
np.sort(df_colo.ECGRUP.unique())

(60887, 30)


array(['I', 'II', 'III', 'IV'], dtype=object)

In [7]:
# Pulmão - C34
pulmao = ['C34']

df_pulmao = data_prep_v2(df_geral, pulmao)
print(df_pulmao.shape)
np.sort(df_pulmao.ECGRUP.unique())

(38646, 30)


array(['I', 'II', 'III', 'IV'], dtype=object)

In [8]:
# Mama - C50
mama = ['C50']

df_mama = data_prep_v2(df_geral, mama)
print(df_mama.shape)
np.sort(df_mama.ECGRUP.unique())

(117338, 30)


array(['I', 'II', 'III', 'IV'], dtype=object)

In [9]:
# Colo de Útero - C53
colo_utero = ['C53']

df_colo_utero = data_prep_v2(df_geral, colo_utero)
print(df_colo_utero.shape)
np.sort(df_colo_utero.ECGRUP.unique())

(23306, 30)


array(['I', 'II', 'III', 'IV'], dtype=object)

In [10]:
# Próstata - C61
prostata = ['C61']

df_prostata = data_prep_v2(df_geral, prostata)
print(df_prostata.shape)
np.sort(df_prostata.ECGRUP.unique())

(94106, 30)


array(['I', 'II', 'III', 'IV'], dtype=object)

In [11]:
# Pele - C44 (Other malignant neoplasms of skin)
pele = ['C44']

df_pele = data_prep_v2(df_geral, pele)
print(df_pele.shape)
np.sort(df_pele.ECGRUP.unique())

(183164, 30)


array(['I', 'II', 'III', 'IV'], dtype=object)

In [19]:
import plotly.graph_objects as go

grup_ec = np.sort(df_pele.ECGRUP.unique())

fig = go.Figure(data=[
    go.Bar(name='Colorretal', x=grup_ec, y=df_colo.ECGRUP.value_counts(normalize=True).sort_index()*100,
           text=df_colo.ECGRUP.value_counts(normalize=True).sort_index()*100),
    go.Bar(name='Pulmão', x=grup_ec, y=df_pulmao.ECGRUP.value_counts(normalize=True).sort_index()*100,
           text=df_pulmao.ECGRUP.value_counts(normalize=True).sort_index()*100),
    go.Bar(name='Mama', x=grup_ec, y=df_mama.ECGRUP.value_counts(normalize=True).sort_index()*100,
           text=df_mama.ECGRUP.value_counts(normalize=True).sort_index()*100),
    go.Bar(name='Colo de Útero', x=grup_ec, y=df_colo_utero.ECGRUP.value_counts(normalize=True).sort_index()*100,
           text=df_colo_utero.ECGRUP.value_counts(normalize=True).sort_index()*100),
    go.Bar(name='Próstata', x=grup_ec, y=df_prostata.ECGRUP.value_counts(normalize=True).sort_index()*100,
           text=df_prostata.ECGRUP.value_counts(normalize=True).sort_index()*100),
    go.Bar(name='Pele', x=grup_ec, y=df_pele.ECGRUP.value_counts(normalize=True).sort_index()*100,
           text=df_pele.ECGRUP.value_counts(normalize=True).sort_index()*100)
])
# Change the bar mode
fig.update_layout(barmode='group', yaxis_title='% Pacientes',
                  xaxis_title='Estadiamento Clínico')
fig.update_traces(texttemplate='%{text:.3s}', textposition='outside')
fig.show()